In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np # linear algebra

In [ ]:
adver = pd.read_csv("../input/advertising-dataset/advertising.csv")
#admissions = admissions.drop('Serial No.',axis = 1)


In [ ]:
adver.head()

In [ ]:
adver.describe()

In [ ]:
X = adver.drop('Sales',axis = 1)
y = adver['Sales']

#X_train,X_val,y_train,y_val = train_test_split(X,y,test_size = .25,random_state = 99)



In [ ]:
X.shape, X.head()

In [ ]:
y.shape, y.head()

## Convert to arrays

In [ ]:
features=np.array(X)

targets=np.array(y).reshape(-1,1)
features [:4,], len(features)

In [ ]:
targets[:4], len(targets)

# normalize feature


In [ ]:
def normalize(features):
    """
    features     -   (200, 3)
    features.T   -   (3, 200)

    We transpose the input matrix, swapping
    cols and rows to make vector math easier
    """
    

    for feature in features.T:
        fmean = np.mean(feature)
        frange = np.amax(feature) - np.amin(feature)

        #Vector Subtraction
        feature -= fmean

        #Vector Division
        feature /= frange

    return features

# make predictions

In [ ]:
def predict(features, weights):
  """
  features - (200, 3)
  weights - (3, 1)
  predictions - (200,1)
  """
  predictions = np.dot(features, weights)
  return predictions

# compute the cost


In [ ]:
def cost_function(features, targets, weights):
    """**
    features:(200,3)
    targets: (200,1)
    weights:(3,1)
    returns average squared error among predictions
    **"""
    N = len(targets)

    predictions = predict(features, weights)

    # Matrix math lets use do this without looping
    sq_error = (predictions - targets)**2
    #print(sq_error.sum())
    # Return average squared error among predictions
    

    return 1.0/(2*N) * sq_error.sum()

# gradient descent

In [ ]:
def update_weights(features, targets, weights, learning_rate):
    '''
    Features:(200, 3)
    Targets: (200, 1)
    Weights:(3, 1)
    '''
    predictions = predict(features, weights)

    #Extract our features
    x1 = features[:,0]
    x2 = features[:,1]
    x3 = features[:,2]

    # Use matrix cross product (*) to simultaneously
    # calculate the derivative for each weight
    d_w1 = -x1*(targets - predictions)
    d_w2 = -x2*(targets - predictions)
    d_w3 = -x3*(targets - predictions)

    # Multiply the mean derivative by the learning rate
    # and subtract from our weights (remember gradient points in direction of steepest ASCENT)
    weights[0][0] -= (learning_rate * np.mean(d_w1))
    weights[1][0] -= (learning_rate * np.mean(d_w2))
    weights[2][0] -= (learning_rate * np.mean(d_w3))

    return weights

# Vectorization

In [ ]:
def update_weights_vectorized(features, targets, weights, learning_rate):
    """
    gradient = X.T * (predictions - targets) / N
    X: (200, 3)
    Targets: (200, 1)
    Weights: (3, 1)
    """
    companies = len(features)

    #1 - Get Predictions
    predictions = predict(features, weights)

    #2 - Calculate error/loss
    error = targets - predictions

    #3 Transpose features from (200, 3) to (3, 200)
    # So we can multiply w the (200,1)  error matrix.
    # Returns a (3,1) matrix holding 3 partial derivatives --
    # one for each feature -- representing the aggregate
    # slope of the cost function across all observations
    gradient = np.dot(-features.T,  error)

    #4 Take the average error derivative for each feature
    gradient /= companies

    #5 - Multiply the gradient by our learning rate
    gradient *= learning_rate

    #6 - Subtract from our weights to minimize cost
    weights -= gradient

    return weights


# train your model

In [ ]:
def train(features, targets, weight,  learning_rate, iters):
    cost_history = []
   
    for i in range(iters):
        weight = update_weights_vectorized(features, targets, weight, learning_rate)
    
    #Calculate cost for auditing purposes
        cost = cost_function(features, targets, weight)
        cost_history.append(cost)
      
        # Log Progress
        #if i ==(iters-1) :
       
        if i% 10 == 0:
            #print("iter={:d}  weight={:.2f}    cost={:.2}".format(i, weight, bias, cost))
            print("iter=" + str(i) + " weight: " + str(weight)  + " cost " +str(cost))#" {:d}    weight={:.2f}    bias={:.4f}    cost={:.2}".format(i, weight, bias, cost))
 
    
    plt.figure()

    plt.plot(np.arange(iters), cost_history,'r')  

    plt.xlabel("Training iterations")
    plt.ylabel("Mean Squared Error")
    plt.title("Error Rate")

    plt.show()

    return  weight, cost_history

# Start implementation of MVLR

In [ ]:
W1 = 0.0
W2 = 0.0
W3 = 0.0
weights = np.array([
    [W1],
    [W2],
    [W3]
])

In [ ]:
normalized_features=normalize(features)
normalized_features[:4,]

In [ ]:
pred = predict(normalized_features, weights)

In [ ]:
cost_function(normalized_features,targets,weights)

In [ ]:
update_weights(normalized_features,targets,weights,0.0005)

In [ ]:
update_weights_vectorized(normalized_features, targets, weights, 0.0005)

In [ ]:
bias = np.ones(shape=(len(features),1))
bfeatures = np.append(bias, normalized_features, axis=1)

W4 = 0.0
weights_bias = np.array([
     [W1],
     [W2],
     [W3],
     [W4]
 ])

In [ ]:
weights_bias,cost_history=train(bfeatures,targets,weights_bias,0.005,iters=10000)

In [ ]:
weights_bias,cost_history=train(bfeatures,targets,weights_bias,0.05,iters=10000)